<a href="https://colab.research.google.com/github/srilamaiti/spring_2023_w266_final_project_heesuk_iris_srila/blob/main/srila/model_embedding_concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install ktrain
!pip install -q pytorch_lightning==1.6.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 29.3 MB/s eta 0:00

In [2]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import tensorflow as tf
import os
import re
import copy

import math
from typing import Optional
import transformers
from ktrain import text

import torch
import torch.utils.checkpoint as checkpoint
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AdamW, DataCollatorWithPadding

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import pickle
import re
from transformers import TrainingArguments, AutoConfig, AutoModelForTokenClassification, DataCollatorForTokenClassification

In [3]:
df = pd.read_csv('train.csv')
df['full_text'] = df["full_text"].replace(re.compile(r'[\n\r\t]'), ' ', regex = True)

label_cols = df.columns[2:]
pred_col_list = ['transformed_pred_' + col for col in label_cols]

orig_df = copy.deepcopy(df)
orig_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal If u change the school policy...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [23]:
split = (0.8, 0.2)
splits = np.multiply(len(orig_df), split).astype(int)
df_train, df_test = orig_df[ : splits[0]], orig_df[splits[0] : ]
y_test = np.array(df_test[label_cols], dtype = "float32")

In [6]:
from transformers import RobertaTokenizer, TFRobertaModel
from transformers import TFBertModel, BertTokenizer
from transformers import TFAutoModel, AutoModel, AutoTokenizer

ROBERTA_MODEL_CHKPT = "roberta-base"
BERT_MODEL_CHKPT = "bert-base-cased"
BERTWEET_MODEL_CHKPT = "vinai/bertweet-base"

bert_model = TFBertModel.from_pretrained(BERT_MODEL_CHKPT)  
bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_CHKPT)

roberta_model = TFBertModel.from_pretrained(ROBERTA_MODEL_CHKPT)  
roberta_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_MODEL_CHKPT)

bertweet_model = TFAutoModel.from_pretrained(BERTWEET_MODEL_CHKPT)
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_MODEL_CHKPT)

In [8]:
MAX_LEN = 512

In [9]:
input_ids = tf.keras.Input(shape = (MAX_LEN ,), dtype = 'int64', name = 'input_ids')
attention_masks = tf.keras.Input(shape = (MAX_LEN ,), dtype = 'int64', name = 'attention_masks')

In [10]:
pretrained_bert_transformer = bert_model([input_ids, attention_masks])
pretrained_roberta_transformer = roberta_model([input_ids, attention_masks])
pretrained_bertweet_transformer = bertweet_model([input_ids, attention_masks])

In [11]:
cls_token_bert = pretrained_bert_transformer[0][:, 0, :]
cls_token_roberta = pretrained_roberta_transformer[0][:, 0, :]
cls_token_bertweet = pretrained_bertweet_transformer[0][:, 0, :]

In [12]:
cls_token_bert

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [13]:
cls_token_roberta

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf.__operators__.getitem_1')>

In [14]:
cls_token_bertweet

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf.__operators__.getitem_2')>

In [17]:
cls_token_concatenated = tf.concat([cls_token_bert, cls_token_roberta, cls_token_bertweet], 0)

In [18]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis = 1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis = -1, keepdims = True)

In [25]:
learning_rate = 1e-5
dropout = .1
epochs = 1
batch_size = 4

In [21]:
output = tf.keras.layers.Dense(6)(cls_token_concatenated)
concat_model = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs = output)
concat_model.compile(tf.keras.optimizers.Adam(learning_rate), loss = MCRMSE, metrics = MCRMSE)

In [26]:
def text_encode(texts, tokenizer, max_len):
    input_ids = []
    # token_type_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, 
                          max_length = max_len, 
                          truncation = True, 
                          padding = 'max_length',
                          add_special_tokens = True)
        input_ids.append(token['input_ids'])
        # token_type_ids.append(token['token_type_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(attention_mask)

In [29]:
train_encoded_input_ids_bert, train_encoded_attention_masks_bert = text_encode(df_train['full_text'], bert_tokenizer, MAX_LEN)
train_encoded_input_ids_roberta, train_encoded_attention_masks_roberta = text_encode(df_train['full_text'], roberta_tokenizer, MAX_LEN)
train_encoded_input_ids_bertweet, train_encoded_attention_masks_bertweet = text_encode(df_train['full_text'], bertweet_tokenizer, MAX_LEN)

In [30]:
len(train_encoded_attention_masks_bert), len(train_encoded_attention_masks_roberta), len(train_encoded_attention_masks_bertweet)

(3128, 3128, 3128)

In [35]:
train_encoded_input_ids_bert[:5]

array([[  101,   146,  1341, ...,     0,     0,     0],
       [  101,  1332,   170, ...,  1106,  1838,   102],
       [  101, 12956,   117, ...,     0,     0,     0],
       [  101,  1109,  1436, ...,  2816,  1272,   102],
       [  101,  6844,  2496, ...,     0,     0,     0]])

In [36]:
train_encoded_input_ids_roberta[:5]

array([[    0,   100,   206, ...,     1,     1,     1],
       [    0,  1779,    10, ...,     7,   192,     2],
       [    0, 23314,     6, ...,     1,     1,     1],
       [    0,   133,   275, ...,   216,    14,     2],
       [    0, 34647,  1760, ...,     1,     1,     1]])

In [37]:
train_encoded_input_ids_bertweet[:5]

array([[    0,     8,   101, ...,     1,     1,     1],
       [    0,   205,    11, ...,     4,   104,     2],
       [    0, 45416,     7, ...,     1,     1,     1],
       [    0,    47,   161, ...,   369,    31,     2],
       [    0,  3997,   951, ...,     1,     1,     1]])

In [ ]:
y_train = np.array(df_train][label_cols], dtype = "float32")

In [ ]:
hist = concat_model.fit([train_encoded_input_ids, train_encoded_attention_masks],
                     y_train,
                     validation_split = .1,
                     batch_size = batch_size,        
                     epochs = epochs
                    )

df_history = pd.DataFrame(hist.history)